# Day 7
## Part 1

Storage is every computer scientist's problem :)

Let's find out the size of each directory through non-binary trees
1. Define a tree for directories

In [1]:
# file objects:
# name:     string for the file name
# size:     interger for the file size 
class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size

# directory tree object:
# name:             name of the node (name of the folder)
# nodes:            dictionary containing its leafs and subtrees
# 
# functions:
# add folder:       adds a folder (of type DirectoryTree) to the tree
#                   folders don't have a size, but they do have a name
# add files:        adds a file (of type File) to the tree like a leaf node
#                   files have a name and a size
# calculate_size:   calculates the size of the tree (i.e. all the files it contains)
#                   recurses through all its subtrees
class DirectoryTree:
    def __init__(self, name):
        self.name = name
        self.nodes = {}

    def add_folder(self, name):
        self.nodes[name] = DirectoryTree(name)

    def add_file(self, name, size):
        self.nodes[name] = File(name, size)

    def calculate_size(self):
        total_size = 0
        for node in self.nodes.keys():
            # add file sizes to the total size
            if type(self.nodes[node]) == File:
                total_size += self.nodes[node].size
            # recurse for non-empty folders
            elif type(self.nodes[node]) == DirectoryTree and self.nodes[node].nodes != []: 
                total_size += self.nodes[node].calculate_size()
        return total_size

In [2]:
dir = DirectoryTree('/')
dir.add_folder('subdir')
dir.add_folder('subdir2')
dir.nodes['subdir2'].add_folder('subsubdir')
dir.nodes['subdir2'].nodes['subsubdir'].add_file('file_1', 11)
dir.nodes['subdir2'].nodes['subsubdir'].add_file('file_2', 12)
print(dir.nodes['subdir2'].calculate_size())

23


2. Import the data

In [3]:
raw_cmd_stuff = open('data/7.txt', 'r')
lines = raw_cmd_stuff.readlines()

3. Iterate through it and build a tree

In [4]:
tree = DirectoryTree('/')
current_dir = tree
parents = []

for line in lines:
    # $ cd / 
    # go to outer directory
    # reset parents
    if line == '$ cd /\n':
        current_dir = tree
        parents = []

    # $ cd .. 
    # move 1 directory down by reconstruction current_dir using parents
    # remove last parent
    elif line == '$ cd ..\n':
        current_dir = tree
        parents.pop()
        for parent in parents:
            current_dir = current_dir.nodes[parent]

    # $ cd something
    # move 1 directory up to something
    # add new parent
    elif line.startswith('$ cd'):
        new_dir = line.split(' ')[-1].replace('\n', '')
        current_dir = current_dir.nodes[new_dir]
        parents.append(new_dir)

    # $ ls
    # list all the files and folders in the current dir
    # (nothing happens here, we simply read in the following lines)
    elif line == '$ ls\n':
        pass

    # dir something
    # add folder something to the current dir
    elif line.startswith('dir'):
        new_folder = line.split(' ')[-1].replace('\n', '')
        current_dir.add_folder(new_folder)

    # number something
    # add file with size and name to current dir
    else:
        size = int(line.split(' ')[0])
        name = line.split(' ')[1].replace('\n', '')
        current_dir.add_file(name, size)


4. Calculate file sizes in all subtrees < 100000

In [5]:
# dictories don't allow duplicates, so just store sizes in array
sizes_all = []
sizes_small = []

def calc_subtree (subtree):
    for node in subtree.nodes:
        if type(subtree.nodes[node]) == DirectoryTree:
            size = subtree.nodes[node].calculate_size()
            sizes_all.append(size)
            if size < 100000:
                sizes_small.append(size)
            calc_subtree(subtree.nodes[node])

calc_subtree(tree)

5. Sum the sizes

In [6]:
sum(sizes_small)

1648397

## Part 2

What folder to remove?

1. Calculate how much needs to be removed
2. Check all sizes that would be large enough and are as small as possible

In [7]:
size_all = tree.calculate_size()
target_size = 40000000
target_remove = size_all - target_size

best_size = size_all

for size in sizes_all:
    # larger than needed
    if size > target_remove:
        # as small as possible
        if size < best_size:
            best_size = size

best_size

1815525